# Four and Five-Factor Models

Carhart's Four-Factor model: The underlying assumption of this extension is that, within a short period of time, a winner stock will remain a winner, while a loser will remain a loser.
An example of a criterion for classifying winners and losers could be the last 12-month cumulative total returns. After identifying the two groups, we long the winners and short
the losers within a certain holding period. The momentum factor (WML; Winners Minus Losers) measures the excess returns of the winner stocks over the loser stocks in the past 12 months

Fama-French's Five-Factor model: Fama and French expanded their three-factor model by adding two factors:
1. Robust Minus Weak (RMW) measures the excess returns of companies with high profit margins (robust profitability) over those with lower profits (weak profitability).
2. Conservative Minus Aggressive (CMA) measures the excess returns of firms with low investment policies (conservative) over those investing more (aggressive).

In [16]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web
import numpy as np

In [6]:
RISKY_ASSET = 'AMZN'
START_DATE = '2013-12-31'
END_DATE = '2018-12-31'

In [7]:
# Download the risk factors from prof. French's website:

# three factors 
df_three_factor = web.DataReader('F-F_Research_Data_Factors', 'famafrench', 
                                 start=START_DATE)[0]
df_three_factor.index = df_three_factor.index.format()

# momentum factor
df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', 
                        start=START_DATE)[0]
df_mom.index = df_mom.index.format()

# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3', 
                                'famafrench', 
                                start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()

In [8]:
asset_df = yf.download(RISKY_ASSET,
                       start=START_DATE,
                       end=END_DATE,
                       adjusted=True,
                       progress=False)

print(f'Downloaded {asset_df.shape[0]} rows of data.')

Downloaded 1259 rows of data.


In [9]:
# Calculate monthly returns

y = asset_df['Adj Close'].resample('M') \
                         .last() \
                         .pct_change() \
                         .dropna()

y.index = y.index.strftime('%Y-%m')
y.name = 'return'

In [25]:
# Merge the datasets for the four-factor models

# join all datasets on the index
four_factor_data = df_three_factor.join(df_mom).join(y)

# rename columns
four_factor_data.columns = ['mkt', 'smb', 'hml', 'rf', 'mom', 'rtn']

# divide everything (except returns) by 100
four_factor_data.loc[:, four_factor_data.columns != 'rtn'] /= 100

# convert index to datetime
# four_factor_data.index = pd.to_datetime(four_factor_data.index, format='%Y-%m')

# select period of interest
four_factor_data = four_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
four_factor_data['excess_rtn'] = four_factor_data.rtn - four_factor_data.rf

four_factor_data.head()

,mkt,smb,hml,rf,mom,rtn,excess_rtn
2014-01,-0.0332,0.0092,-0.0202,0.0,0.0171,-0.100554,-0.100554
2014-02,0.0465,0.0037,-0.0031,0.0,0.0207,0.009507,0.009507
2014-03,0.0043,-0.0187,0.0492,0.0,-0.0329,-0.071058,-0.071058
2014-04,-0.0019,-0.0420,0.0114,0.0,-0.0389,-0.095847,-0.095847
2014-05,0.0206,-0.0189,-0.0013,0.0,0.0088,0.027685,0.027685


In [27]:
# Merge the datasets for the five-factor models

# join all datasets on the index
five_factor_data = df_five_factor.join(y)

# rename columns
five_factor_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma', 'rf', 'rtn']

# divide everything (except returns) by 100
five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100

# select period of interest
five_factor_data = five_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf

five_factor_data.head()

,mkt,smb,hml,rmw,cma,rf,rtn,excess_rtn
2014-01,-0.0332,0.0059,-0.0202,-0.0388,-0.0143,0.0,-0.100554,-0.100554
2014-02,0.0465,0.0016,-0.0031,-0.0023,-0.0048,0.0,0.009507,0.009507
2014-03,0.0043,-0.0113,0.0492,0.0211,0.0198,0.0,-0.071058,-0.071058
2014-04,-0.0019,-0.0413,0.0114,0.0345,0.0103,0.0,-0.095847,-0.095847
2014-05,0.0206,-0.0189,-0.0013,0.0005,-0.0101,0.0,0.027685,0.027685


In [28]:
# Estimate the four-factor model

four_factor_model = smf.ols(formula='excess_rtn ~ mkt + smb + hml + mom', 
                            data=four_factor_data).fit()

print(four_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.520
Method:                 Least Squares   F-statistic:                     16.99
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           4.00e-09
Time:                        15:46:29   Log-Likelihood:                 86.454
No. Observations:                  60   AIC:                            -162.9
Df Residuals:                      55   BIC:                            -152.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0093      0.008      1.158      0.2

In [29]:
# Estimate the five-factor model

five_factor_model = smf.ols(
    formula='excess_rtn ~ mkt + smb + hml + rmw + cma', 
    data=five_factor_data
).fit()

print(five_factor_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     16.05
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           1.13e-09
Time:                        15:46:32   Log-Likelihood:                 89.647
No. Observations:                  60   AIC:                            -167.3
Df Residuals:                      54   BIC:                            -154.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0099      0.008      1.296      0.2

According to the five-factor model, Amazon's excess returns are negatively
exposed to most of the factors (all but the market factor). Here, we present an
example of the interpretation of the coefficients: an increase by 1 percentage point
in the market factor results in an increase of 0.015 p.p. In other words, for a 1%
return by the market factor, we can expect our portfolio (Amazon's stock) to
return 1.5508 * 1% in excess of the risk-free rate.